In [3]:
import csv 
import logging
import re
import sqlite3 as sq
import functions_package as fp
from datetime import datetime
currentDT = datetime.now().strftime("%d%m%Y%H%M%S")

#Creating logfilename with current date and time
log_filename = 'LogFile_'+str(currentDT)+'.log'
logging.basicConfig(filename=log_filename,level = logging.INFO, format = '%(asctime)s %(levelname)s %(message)s')

logging.info("...Program starts....")   

#Fetching the input path from user
location = input("Enter the location of files  : ")
logging.info(f"Fetching the input path from user. Path entered by user : {location}")


#creating an object for the class inside the package
gt_cnt = fp.packageClass(location)

#Getting the list of text files from packageClass class
file_names = gt_cnt.get_txt_files()
logging.info(f"List of text files present in the given path : {file_names}")
try:
    if file_names == []:
        raise Exception ("No text file found")
except Exception as e1:
    logging.error(f"Exception found : {e1}")


def qn1():
    """ This function tries to find out a count of each and every word in a respective file and return a list of tuple with word and 
its respective count  """
    logging.info("...Execution started for Qn1...")
    try:
        for file_name in file_names :
            with open (location+'\\'+ file_name,"r",encoding = 'utf-8') as file:
                data = csv.reader(file,delimiter='\n')

                logging.info(f"Reading {file_name}")

                #Calling function 'get_list_of_data' from the class 'packageClass' to get the list of all 
                #the words present in the text file in sorted form
                list_of_data = gt_cnt.get_list_of_data(data)
                logging.info(f"Fetching sorted list of all the words from {file_name}")

                #Calling function 'get_distinct_data' from the class 'packageClass' to get the list of all 
                #distinct words present in list_of_data in sorted form
                distinct_data  = gt_cnt.get_distinct_data(list_of_data)
                logging.info("Fetching distinct words from the sorted list_of_data")
                
                #Calling function 'get_count_of_words' from the class 'packageClass' to get the count of 
                #every word in distinct_data from list_of_data 
                count_of_words = gt_cnt.get_count_of_words(list_of_data,distinct_data)
                logging.info("Finding the count of words")

              
                #Using zip function to create list of tuples containg the word and it's count of data
                globals()[f"outputq1_{file_name.replace('.','_')}"] = list(zip(distinct_data,count_of_words))
                logging.info(f"Creating a list of tuple of each words and its count ")
                logging.info(f"Output stored in outputq1_{file_name.replace('.','_')}")
  
                #Closing the file
                file.close()

        logging.info("...Execution completed for Qn1...")
        
    except Exception as excep:
        logging.error(f"Exception occured: {excep}")
        
logging.shutdown() 

        
def qn2():
    """This function tries to perform a reduce operation to get a count of all the word starting with same alphabet """
    try:
        logging.info("...Execution started for Qn2...")
        for file_name in file_names :
            with open (location+'\\'+ file_name,"r",encoding = 'utf-8') as file:
                data = csv.reader(file,delimiter='\n')
                
                logging.info(f"Reading {file_name}")

                #Calling function 'get_list_of_data' from the class 'packageClass' to get the list of all the 
                #words present in the text file in sorted form
                list_of_data = gt_cnt.get_list_of_data(data)
                logging.info(f"Fetching sorted list of all the words from {file_name}")

                #storing all alphabets in a list
                alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

                def countfunc(a):
                    """This function counts the number of words starting with the character passed"""
                    count=0
                    for i in list_of_data:
                        if i[0].lower().startswith(a):
                            count+=1
                    return count

                #Using map function to find the count of words present in list_of_data for each english alphabets
                count_of_alph = list(map(countfunc, alphabets))
                logging.info("Finding the count of words for each alphabet")

                #Using zip function to create list of tuples containg the alphabets and it's count of data
                globals()[f"outputq2_{file_name.replace('.','_')}"] = list(zip(alphabets,count_of_alph))
                logging.info(f"Creating a list of tuple of each alphabet and its count ")
                logging.info(f"Output stored in outputq2_{file_name.replace('.','_')}")
                
                #Closing the file
                file.close()

        logging.info("...Execution completed for Qn2...")
        
    except Exception as excep:
        logging.error(f"Exception occured: {excep}")
    
logging.shutdown() 

    
def qn3():
    """ This function tries to filter out all the words from dataset"""
    try:
        logging.info("...Execution started for Qn3...")
        for file_name in file_names :
            with open (location+'\\'+ file_name,"r",encoding = 'utf-8') as file:
                data = csv.reader(file,delimiter='\n')
                
                logging.info(f"Reading {file_name}")

                #Calling function 'get_list_of_data' from the class 'packageClass' to get the list of 
                #all the words present in the text file in sorted form
                list_of_data = gt_cnt.get_list_of_data(data)
                logging.info(f"Fetching sorted list of all the words from {file_name}")

                #Getting words only from each of the element in list_of_data
                words=[]
                for i in list_of_data:
                    words.append("".join(re.split("[^a-zA-Z]*", i)))
                
                
                #Using filter to remove the blank elements in words list
                def filterblank(a):
                    """This function return only non blank elements in a list"""
                    if a != '':
                        return a
                
                filtered_words = list(filter(filterblank,words))                
                
                #Using zip function to create list of tuples containg the alphabets and it's count of data
                globals()[f"outputq3_{file_name.replace('.','_')}"] = filtered_words
                logging.info(f"Creating a list of all the filtered words present in the file")
                logging.info(f"Output stored in outputq3_{file_name.replace('.','_')}")

                #Closing the file
                file.close()

        logging.info("...Execution completed for Qn3...")
        
    except Exception as excep:
        logging.error(f"Exception occured: {excep}")

    
logging.shutdown()                                                                                                            
    

def qn4():
    """This function creates a tuple set of all the records avaialble in all the five file and then store it in sqllite DB """
    try:
        logging.info("...Execution started for Qn4...")
        var_name=[]
        for file_name in file_names :
            with open (location+'\\'+ file_name,"r",encoding = 'utf-8') as file:
                data = csv.reader(file,delimiter='\n')
                logging.info(f"Reading {file_name}")

                #Calling function 'get_list_of_data' from the class 'get_count_q1' to get the list of all the words present in the text file in sorted form
                list_of_data = gt_cnt.get_list_of_data(data)
                list_of_data = set(list_of_data)
                logging.info(f"Fetching distinct list of all the words from {file_name}")
                
                globals()[f"list_of_data_{file_name.replace('.','_')}"] = list_of_data
                var = globals()[f"list_of_data_{file_name.replace('.','_')}"]
                var_name.append(var)
                logging.info(f"list of data for {file_name} stored in list_of_data_{file_name.replace('.','_')}")
                
                #Closing the file
                file.close()

    except Exception as excep:
        logging.error(f"Exception occured: {excep}")
    
    
    try:
        if(var_name[0]&var_name[1]&var_name[2]&var_name[3]&var_name[4]):
            globals()["outputq4"] = list(var_name[0]&var_name[1]&var_name[2]&var_name[3]&var_name[4])
            logging.info("Common words in all 5 files are stored in outputq4")
            logging.info(f"Count of common words  : {len(outputq4)}")
            #Inserting data in sqlite db
            insert_into_db(outputq4)
            logging.info("...Execution completed for Qn4...")
        else:
            logging.warning("No common words found") 
    except Exception as exp:
        logging.error(f"Exception : {exp}")
           
    
    
    
    
logging.shutdown()  




def insert_into_db(outputq4):
    """This function inserts the output of q4 into sqlite db"""
    
    #Establishing the sqlite connection
    mydb = sq.connect("test_db.db")    #creating sqlite3 connection
    cursor = mydb.cursor()
    logging.info("Successfully Connected to SQLite")
    try: 
        cursor.execute("create table tb_common_words (common_words text) ")
        for i in range(len(outputq4)):
            cursor.execute('INSERT INTO tb_common_words(common_words) VALUES (?);', [(outputq4[i])])
        mydb.commit()
        logging.info("Data in outputq4 inserted successfully into the table")
                
    except Exception as f:
        logging.error(f"Exception occured : {f}. Unable to insert data")
    
    except sqlite3.Error as error:
        logging.error("Failed to insert data into sqlite table", error)
    
    finally:
        if mydb:
            mydb.close()
            logging.info("The SQLite connection is closed")
    
#Function calling    
qn1()  
qn2()
qn3()
qn4()

logging.info("...Program ends...")

Enter the location of files  : C:\Users\remya\Downloads\Homework
